## Publish a Translator edge to the Knowledge Collaboratory

https://github.com/fair-workflows/Nanopub

Install `rdflib` and `nanopub` packages:

```bash
pip install -r requirements.txt
```

You will need to setup the Nanopublication client to publish, run this interactive command to do so:

```bash
setup_nanopub_profile
```

> This should create a `.nanopub` folder in your home folder with your RSA key and a `profile.yml`

In [33]:
edge_to_publish = {
  "association_type": "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation",
  "has_confidence_level": 0.6970851597240628,
  "object": "DRUGBANK:DB00394",
  "predicate": "biolink:treated_by",
  "provided_by": "Translator OpenPredict model openpredict-baseline-omim-drugbank",
  "subject": "OMIM:601626"
}
comment = 'This statement is false'
negated = True
evidence = 'https://pubmed.ncbi.nlm.nih.gov/3159979/'

In [34]:
import json 
import requests
from nanopub import Publication, NanopubClient
from rdflib import Graph, Namespace, URIRef, Literal, RDF, FOAF, RDFS, XSD
BIOLINK = Namespace("https://w3id.org/biolink/vocab/")
NP = Namespace("http://purl.org/nanopub/temp/mynanopub#")
nanopub_rdf = Graph()
nanopub_rdf.bind("biolink", URIRef('https://w3id.org/biolink/vocab/'))

# Add edge data
edge_uri = NP['edge']
# nanopub_rdf.add( (edge_uri, RDF.type, RDF.Statement ) )
for prop, value in edge_to_publish.items():
    nanopub_rdf.add( (edge_uri, BIOLINK[prop], Literal(value) ) )

# Add user comment
comment_uri = NP['comment']
nanopub_rdf.add( (comment_uri, RDF.type, RDF.Statement ) )
nanopub_rdf.add( (comment_uri, RDFS.seeAlso, edge_uri ) )
nanopub_rdf.add( (comment_uri, RDFS.comment, Literal(comment) ) )
nanopub_rdf.add( (comment_uri, BIOLINK['negated'], Literal(negated,datatype=XSD.boolean) ) )
nanopub_rdf.add( (comment_uri, BIOLINK['has_evidence'], URIRef(evidence) ) )

# print(nanopub_rdf.serialize(format='turtle'))
publication = Publication.from_assertion(assertion_rdf=nanopub_rdf)
print(publication)

Original source URI = None
@prefix : <http://purl.org/nanopub/temp/mynanopub#> .
@prefix biolink: <https://w3id.org/biolink/vocab/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:assertion {
    :comment a rdf:Statement ;
        rdfs:comment "" ;
        rdfs:seeAlso :edge ;
        biolink:has_evidence <> ;
        biolink:negated true .

    :edge biolink:association_type "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation" ;
        biolink:has_confidence_level 6.970852e-01 ;
        biolink:object "DRUGBANK:DB00394" ;
        biolink:predicate "biolink:treated_by" ;
        biolink:provided_by "Translator OpenPredict model openpredict-baseline-omim-drugbank" ;
        biolink:subject "OMIM:601626" .
}

:pubInfo {
    : prov:generatedAtTime "2021-02-02T21:40:24.9

In [35]:
# Create the client, that allows publishing nanopubs
np_client = NanopubClient()

In [36]:
# publication_info = np_client.publish(publication)